In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
ratings = {}
k = 28
home_advantage = 22
season_home = 0
season_away = 0
total_home = 0
total_away = 0
one_zero = [0,0]
two_zero = [0,0]
two_one = [0,0]
one_one = [0,0]
two_two = [0,0]
home_10 = [0,0]
away_10 = [0,0]
home_20 = [0,0]
away_20 = [0,0]
home_21 = [0,0]
away_21 = [0,0]

In [3]:
ratings['현대캐피탈'] = 1500
ratings['삼성화재'] = 1500
ratings['KB손해보험'] = 1500
ratings['한국전력'] = 1500
ratings['대한항공'] = 1500
ratings['상무'] = 1500
ratings['우리카드'] = 1500
ratings['OK금융그룹'] = 1500

In [4]:
for season in range(1,19):
    season_home = 0
    season_away = 0
    if season <10:
        season = "0"+str(season)
    for game_number in range(1,400):
        url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=0"+str(season)+"&g_part=201&r_round=1&g_num="+str(game_number)+"&"
        res = requests.get(url)
        print(url)
        soup = BeautifulSoup(res.text,"lxml")
        team_names = soup.find_all("span",attrs={"class":"team"}) #팀 이름을 리스트로 가져옴
        if team_names == []:
            for key in ratings:
                if ratings[key] != 0:
                    ratings[key] = ratings[key]+(1500-ratings[key])/2
            break
        results = soup.find_all("p",attrs={"class":re.compile("^result")}) #승패 결과를 리스트로 가져옴
        points = soup.find_all("p",attrs={"class":"num"}) #세트 스코어를 리스트로 가져옴
        if points == []:
            break
        if season == 18:
            if points[0].get_text() == points[1].get_text():
                break
        home_team_name = team_names[0].get_text()
        away_team_name = team_names[1].get_text()
        if home_team_name == "LG화재":
            home_team_name = "KB손해보험"
        if away_team_name == "LG화재":
            away_team_name = "KB손해보험"
        if home_team_name == "LIG":
            home_team_name = "KB손해보험"
        if away_team_name == "LIG":
            away_team_name = "KB손해보험"
        if home_team_name == "LIG손해보험":
            home_team_name = "KB손해보험"
        if away_team_name == "LIG손해보험":
            away_team_name = "KB손해보험"
        if home_team_name == "KEPCO45":
            home_team_name = "한국전력"
        if away_team_name == "KEPCO45":
            away_team_name = "한국전력"
        if home_team_name == "KEPCO":
            home_team_name = "한국전력"
        if away_team_name == "KEPCO":
            away_team_name = "한국전력"
        if home_team_name == "신협상무":
            home_team_name = "상무"
        if away_team_name == "신협상무":
            away_team_name = "상무"
        if home_team_name == "우리캐피탈":
            home_team_name = "우리카드"
        if away_team_name == "우리캐피탈":
            away_team_name = "우리카드"
        if home_team_name == "상무신협":
            home_team_name = "상무"
        if away_team_name == "상무신협":
            away_team_name = "상무"
        if home_team_name == "드림식스":
            home_team_name = "우리카드"
        if away_team_name == "드림식스":
            away_team_name = "우리카드"
        if home_team_name == "러시앤캐시":
            home_team_name = "OK금융그룹"
        if away_team_name == "러시앤캐시":
            away_team_name = "OK금융그룹"
        if home_team_name == "OK저축은행":
            home_team_name = "OK금융그룹"
        if away_team_name == "OK저축은행":
            away_team_name = "OK금융그룹"
            
        if home_team_name not in ratings:
            continue
        if away_team_name not in ratings:
            continue

        results = soup.find_all("p",attrs={"class":re.compile("^result")}) #승패 결과를 리스트로 가져옴
        points = soup.find_all("p",attrs={"class":"num"}) #세트 스코어를 리스트로 가져옴
        
        if points == []: #게임이 없으면 시즌을 종료함
            break
        if season == 18: #18시즌이 끝나면 프로그램을 종료함
            if points[0].get_text() == points[1].get_text():
                break
        #모든 확인이 끝난 지점
        teams = soup.find_all("div",{"class":"wrp_lst"})
        for team in teams:
            data = []
            record = team.find("table",{"class":"lst_board lst_scroll w837 record_1"})
        expected_score = 1/(1+10**((ratings[away_team_name]-ratings[home_team_name]-home_advantage)/400)) #예상점수
        
        #세트당 점수를 가져옴
        home_set_score = soup.find("table",{"class":"inner_table"}).find("tbody").find_all("tr")[0].find_all("td")[1:-1]
        away_set_score = soup.find("table",{"class":"inner_table"}).find("tbody").find_all("tr")[1].find_all("td")[1:-1]

        if int(points[0].get_text()) > int(points[1].get_text()): #홈팀 승리시 레이팅 조정
            total_home += 1
            season_home += 1
            ratings[home_team_name] = ratings[home_team_name]+k*(1-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(1-expected_score)

            current_set = [0,0]
            for i in range(len(home_set_score)):
                if int(home_set_score[i].get_text()) > int(away_set_score[i].get_text()):
                    current_set[0]+=1
                elif int(home_set_score[i].get_text()) < int(away_set_score[i].get_text()):
                    current_set[1]+=1

                if current_set == [1,0]:
                    one_zero[0]+=1
                    home_10[0]+=1
                elif current_set == [2,0]:
                    two_zero[0]+=1
                    home_20[0]+=1
                elif current_set == [0,1]:
                    one_zero[1]+=1
                    away_10[1]+=1
                elif current_set == [0,2]:
                    two_zero[1]+=1
                    away_20[1]+=1
                elif current_set == [2,1]:
                    two_one[0]+=1
                    home_21[0]+=1
                elif current_set == [1,2]:
                    two_one[1]+=1
                    away_21[1]+=1
                elif current_set == [1,1]:
                    one_one[0]+=1
                elif current_set == [2,2]:
                    two_two[0]+=1

        else: #홈팀 패배시 레이팅 조정
            ratings[home_team_name] = ratings[home_team_name]+k*(0-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(0-expected_score)
            total_away += 1
            season_away += 1

            current_set = [0,0]
            for i in range(len(home_set_score)):
                if int(home_set_score[i].get_text()) > int(away_set_score[i].get_text()):
                    current_set[0]+=1
                elif int(home_set_score[i].get_text()) < int(away_set_score[i].get_text()):
                    current_set[1]+=1

                if current_set == [1,0]:
                    one_zero[1]+=1
                    home_10[1]+=1
                elif current_set == [2,0]:
                    two_zero[1]+=1
                    home_20[1]+=1
                elif current_set == [0,1]:
                    one_zero[0]+=1
                    away_10[0]+=1
                elif current_set == [0,2]:
                    two_zero[0]+=1
                    away_20[0]+=1
                elif current_set == [2,1]:
                    two_one[1]+=1
                    home_21[1]+=1
                elif current_set == [1,2]:
                    two_one[0]+=1
                    away_21[0]+=1
                elif current_set == [1,1]:
                    one_one[1]+=1
                elif current_set == [2,2]:
                    two_two[1]+=1

https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=1&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=2&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=3&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=4&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=5&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=6&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=7&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=8&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=9&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1

In [5]:
sorted_dict = sorted(ratings.items(), key = lambda item: item[0], reverse = True)
print(sorted_dict)

[('현대캐피탈', 1452.5979460375388), ('한국전력', 1488.3093830654566), ('우리카드', 1505.2707906593132), ('상무', 1499.744901817001), ('삼성화재', 1457.7882766297016), ('대한항공', 1588.2090658594811), ('OK금융그룹', 1504.0376794063895), ('KB손해보험', 1504.0419565251182)]


In [6]:
def xwin_rate(A,B):
    return 1/(1+10**((B-A-22)/400))

In [7]:
xwin_rate(ratings['현대캐피탈'], ratings['한국전력'])

0.48027789367986334

In [8]:
1/2.55

0.3921568627450981

In [9]:
print("total home winrate : ",total_home,total_away,total_home/(total_away+total_home))
print("1:0 winrate - ",one_zero[0],one_zero[1],one_zero[0]/(one_zero[1]+one_zero[0]))
print("2:0 winrate - ",two_zero[0],two_zero[1],two_zero[0]/(two_zero[1]+two_zero[0]))
print("2:1 winrate - ",two_one[0],two_one[1],two_one[0]/(two_one[1]+two_one[0]))
print("1:1 home winrate - ",one_one[0],one_one[1],one_one[0]/(one_one[0]+one_one[1]))
print("2:2 home winrate - ",two_two[0],two_two[1],two_two[0]/(two_two[0]+two_two[1]))
print("1:0 home winrate - ",home_10[0],home_10[1],home_10[0]/(home_10[0]+home_10[1]))
print("2:0 home winrate - ",home_20[0],home_20[1],home_20[0]/(home_20[0]+home_20[1]))
print("2:1 home winrate - ",home_21[0],home_21[1],home_21[0]/(home_21[0]+home_21[1]))
print("1:0 away winrate - ",away_10[0],away_10[1],away_10[0]/(away_10[0]+away_10[1]))
print("2:0 away winrate - ",away_20[0],away_20[1],away_20[0]/(away_20[0]+away_20[1]))
print("2:1 away winrate - ",away_21[0],away_21[1],away_21[0]/(away_21[0]+away_21[1]))

total home winrate :  1066 907 0.5402939685757729
1:0 winrate -  1499 474 0.7597567156614293
2:0 winrate -  1132 86 0.9293924466338259
2:1 winrate -  865 249 0.7764811490125674
1:1 home winrate -  381 374 0.5046357615894039
2:2 home winrate -  230 231 0.49891540130151846
1:0 home winrate -  814 222 0.7857142857142857
2:0 home winrate -  638 39 0.9423929098966026
2:1 home winrate -  446 118 0.7907801418439716
1:0 away winrate -  685 252 0.7310565635005336
2:0 away winrate -  494 47 0.9131238447319778
2:1 away winrate -  419 131 0.7618181818181818
